In [1]:
!pip install keras
!pip install lightgbm

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
Looking in indexes: http://mirrors.tencentyun.com/pypi/simple


In [2]:
import pandas as pd
import numpy as np

In [3]:
train=pd.read_csv("train_all.csv")
test=pd.read_csv("test_all.csv")
all=pd.concat([train,test],ignore_index=True)

In [4]:
item=all["user_items"].values.tolist()

In [5]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
docs = item

# 只考虑最常见的词
max_words = 3000
# 统一的序列化长度
# 大于这个长度截断，小于这个长度用0补全
maxlen = 20
# 嵌入的维度
embedding_dim = 64

# 分词
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(docs)
# 字典
word_index = tokenizer.word_index

# 序列化
sequences = tokenizer.texts_to_sequences(docs)
# 统一序列长度
data = pad_sequences(sequences, maxlen=maxlen)

# Embedding模型
model = Sequential()
# Embedding至少需要max_wrods和embedding_dim两个参数
model.add(Embedding(max_words, embedding_dim, input_length=maxlen, name='embedding'))
model.compile('rmsprop', 'mse')

out = model.predict(data)
# print(out)
# print(out.shape)
# # 查看权重
# layer = model.get_layer('embedding')
# print(layer.get_weights())

embedding_list=[]
for i in out:
    temp=sum(i)/10
    temp=temp.tolist()
    embedding_list.append(temp)

Using TensorFlow backend.


In [6]:
train_list=embedding_list[0:900000]
test_list=embedding_list[900000:1900000]

In [7]:
# a=pd.DataFrame(embedding_list)

In [8]:
# train.drop(["user_items"],axis=1,inplace=True)

In [9]:
# train_em=pd.concat([train,a],axis=1)

In [10]:
# train_em.to_csv("train_em.csv")

In [11]:
import lightgbm as lgb
from sklearn import datasets
from sklearn.model_selection import train_test_split
target=np.array(train["age"])#年龄建模
X_train,X_test,y_train,y_test=train_test_split(train_list,target,test_size=0.03)
import numpy as np
train_data=lgb.Dataset(X_train,label=y_train)
validation_data=lgb.Dataset(X_test,label=y_test)
params={
    'learning_rate':0.1,
    'lambda_l1':0.1,
    'lambda_l2':0.2,
    'max_depth':4,
    'objective':'multiclass',
    'num_class':13,  #lightgbm.basic.LightGBMError: b‘Number of classes should be specified and greater than 1 for multiclass training‘
}
clf=lgb.train(params,train_data,valid_sets=[validation_data])
clf.save_model('age_model.txt')
from sklearn.metrics import roc_auc_score,accuracy_score
y_pred=clf.predict(X_test)
y_pred=[list(x).index(max(x)) for x in y_pred]
# print(y_pred)
print(accuracy_score(y_test,y_pred))

[1]	valid_0's multi_logloss: 2.03845
[2]	valid_0's multi_logloss: 2.03763
[3]	valid_0's multi_logloss: 2.0369
[4]	valid_0's multi_logloss: 2.03621
[5]	valid_0's multi_logloss: 2.03548
[6]	valid_0's multi_logloss: 2.03487
[7]	valid_0's multi_logloss: 2.03425
[8]	valid_0's multi_logloss: 2.03366
[9]	valid_0's multi_logloss: 2.03302
[10]	valid_0's multi_logloss: 2.03244
[11]	valid_0's multi_logloss: 2.0319
[12]	valid_0's multi_logloss: 2.03136
[13]	valid_0's multi_logloss: 2.03087
[14]	valid_0's multi_logloss: 2.03035
[15]	valid_0's multi_logloss: 2.02986
[16]	valid_0's multi_logloss: 2.02938
[17]	valid_0's multi_logloss: 2.02891
[18]	valid_0's multi_logloss: 2.02847
[19]	valid_0's multi_logloss: 2.02803
[20]	valid_0's multi_logloss: 2.0276
[21]	valid_0's multi_logloss: 2.0272
[22]	valid_0's multi_logloss: 2.02682
[23]	valid_0's multi_logloss: 2.02639
[24]	valid_0's multi_logloss: 2.02602
[25]	valid_0's multi_logloss: 2.02563
[26]	valid_0's multi_logloss: 2.02525
[27]	valid_0's multi_logl

In [12]:
import lightgbm as lgb
from sklearn import datasets
from sklearn.model_selection import train_test_split
target=np.array(train["gender"])#性别建模
X_train,X_test,y_train,y_test=train_test_split(train_list,target,test_size=0.04)
import numpy as np
train_data=lgb.Dataset(X_train,label=y_train)
validation_data=lgb.Dataset(X_test,label=y_test)
params={
    'learning_rate':0.1,
    'lambda_l1':0.1,
    'lambda_l2':0.2,
    'max_depth':4,
    'objective':'multiclass',
    'num_class':4,  #lightgbm.basic.LightGBMError: b‘Number of classes should be specified and greater than 1 for multiclass training‘
}

clf=lgb.train(params,train_data,valid_sets=[validation_data])
clf.save_model('gender_model.txt')
from sklearn.metrics import roc_auc_score,accuracy_score
y_pred=clf.predict(X_test)
y_pred=[list(x).index(max(x)) for x in y_pred]
# print(y_pred)
print(accuracy_score(y_test,y_pred))

[1]	valid_0's multi_logloss: 0.631233
[2]	valid_0's multi_logloss: 0.629418
[3]	valid_0's multi_logloss: 0.627836
[4]	valid_0's multi_logloss: 0.626366
[5]	valid_0's multi_logloss: 0.625129
[6]	valid_0's multi_logloss: 0.623899
[7]	valid_0's multi_logloss: 0.622739
[8]	valid_0's multi_logloss: 0.621654
[9]	valid_0's multi_logloss: 0.620745
[10]	valid_0's multi_logloss: 0.619826
[11]	valid_0's multi_logloss: 0.618807
[12]	valid_0's multi_logloss: 0.618143
[13]	valid_0's multi_logloss: 0.617435
[14]	valid_0's multi_logloss: 0.616738
[15]	valid_0's multi_logloss: 0.616165
[16]	valid_0's multi_logloss: 0.615419
[17]	valid_0's multi_logloss: 0.6148
[18]	valid_0's multi_logloss: 0.614284
[19]	valid_0's multi_logloss: 0.613591
[20]	valid_0's multi_logloss: 0.612946
[21]	valid_0's multi_logloss: 0.612507
[22]	valid_0's multi_logloss: 0.611883
[23]	valid_0's multi_logloss: 0.611442
[24]	valid_0's multi_logloss: 0.610879
[25]	valid_0's multi_logloss: 0.610499
[26]	valid_0's multi_logloss: 0.6100

In [13]:
import lightgbm as lgb
# 模型加载
gbm = lgb.Booster(model_file='age_model.txt')
# 模型预测
age_pred = gbm.predict(test_list)
age_pred=[list(x).index(max(x)) for x in age_pred]

In [14]:
import lightgbm as lgb
# 模型加载
gbm = lgb.Booster(model_file='gender_model.txt')
# 模型预测
gender_pred = gbm.predict(test_list, num_iteration=gbm.best_iteration)
gender_pred=[list(x).index(max(x)) for x in gender_pred]

In [15]:
submission=pd.DataFrame(test["user_id"])

In [16]:
submission["predicted_age"]=age_pred
submission["predicted_gender"]=gender_pred

In [ ]:
submission.to_csv("submission_5 29.csv",index=False)

In [ ]:
submission